In [ ]:
import sycamore
import os
from sycamore.transforms.partition import ArynPartitioner
from sycamore.transforms.extract_schema import (
    OpenAIPropertyExtractor,
)
from sycamore.llms import OpenAI, OpenAIModels
from sycamore.data.element import Element
from sycamore.data.document import Document
from sycamore.functions import HuggingFaceTokenizer, OpenAITokenizer
from sycamore.llms import OpenAI, OpenAIModels
from sycamore.transforms.embed import SentenceTransformerEmbedder
from sycamore.llms.prompts.default_prompts import LlmFilterMessagesJinjaPrompt
from sycamore.llms.prompts.prompts import JinjaPrompt
from sycamore.transforms.extract_entity import OpenAIEntityExtractor
from sycamore.query.execution.operations import summarize_data
from sycamore.transforms.summarize import MultiStepDocumentSummarizer, EtCetera, OneStepDocumentSummarizer
from sycamore.llms.llms import LLMMode

In [ ]:
context = sycamore.init()
initial_docset = (context.read
                  .aryn(aryn_url="https://test-api.aryn.ai/v1/storage", docset_id="<YOUR-DOCSET-ID>", aryn_api_key="<YOUR-ARYN-API-KEY>"))

In [ ]:
def remove_original_elements(doc: Document):
    del doc.properties['_original_elements']
    return doc

def filter_parent_documents(doc:Document):
    return "parent_id" in doc
    

exploded_docset2 = initial_docset.map(remove_original_elements).spread_properties(["earnings_calls"]).explode().filter(filter_parent_documents)
removed_orig_docset = initial_docset.map(remove_original_elements).spread_properties(["earnings_calls"])

In [ ]:
from sycamore.llms import OpenAI, OpenAIModels
oai = OpenAI(OpenAIModels.GPT_4O)

In [ ]:
## Cell to answer question: tell me the number of customers MongoDB had at the end of Q1
entity_extractor = OpenAIEntityExtractor(entity_name="num_customers", llm=oai, num_of_elements=10,
                      field = "text_representation", use_elements=True)
                      

mdb_docset = removed_orig_docset.filter( lambda doc: doc.properties['earnings_calls']['company_ticker'] == 'MDB' and doc.properties['earnings_calls']['quarter']=='Q1').extract_entity(entity_extractor)

In [ ]:
mdb_docset.take(1)[0].properties.get("num_customers")

In [ ]:
avgo_docset = removed_orig_docset.filter( lambda doc: doc.properties['earnings_calls']['company_ticker'] == 'AVGO').llm_filter(new_field="_autogen_LLMFilterOutput",
            prompt=LlmFilterMessagesJinjaPrompt.fork(filter_question="Does this mention the VMWare acquisition?", use_elements=True),
            field = "text_representation",
            llm=oai,
            keep_none=True)

In [ ]:
## Cell to answer the question tell me the first earnings call where the VMWare acquisiton was mentioned
from sycamore.transforms import DateTimeStandardizer

def filterVMware(elem: Element) -> bool:
  return "VMware" in elem.text_representation

vwmare_docset_sorted = (removed_orig_docset
                        .filter(lambda doc: doc.properties['earnings_calls']['company_ticker'] == 'AVGO')
                        .filter_elements(filterVMware)
                        .map(lambda doc: DateTimeStandardizer.standardize(doc, key_path = ["properties","earnings_calls","date"]))
                        .sort(descending=False, field="properties.earnings_calls.dateTime"))
vwmare_docset_sorted.take(1)[0].properties['earnings_calls']['day']

In [ ]:
## Cell to answer question summarize the impact of the VMWare acquisiton on Broadcom's earnings
myprompt = JinjaPrompt(
    system="You are a helpful financial assistant.",
    user="""Here's an earnings call. Please answer the question {{ question }}
            {% for elt in doc.elements %}
            {{ elt.text_representation }}
            {% endfor %}""",
    question="Summarize the impact of the vmware acquisition on broadcom's earnings"
)

vmware_acquistion_summary = (removed_orig_docset.filter(lambda doc: doc.properties['earnings_calls']['company_ticker'] == 'AVGO')
                     .filter_elements(filterVMware)
                     .llm_map(prompt=myprompt, output_field="acquisition_impact_summary", llm=oai, llm_mode=LLMMode.ASYNC))


In [ ]:
question = "Summarize the impact of the vmware acquisition on broadcom's earnings. Give a 2 line summary for each quarter."
data_desc = "Broadcom earnings calls"

oai = OpenAI(OpenAIModels.GPT_4O)
oaitk = OpenAITokenizer(OpenAIModels.GPT_4O.value.name, max_tokens=100_000)
summarizer = MultiStepDocumentSummarizer(
    llm=oai, 
    llm_mode=LLMMode.ASYNC, 
    question=question, 
    data_description=data_desc, 
    fields=['acquisition_impact_summary'],
    tokenizer=oaitk
)
summarize_data(
    llm=oai,
    question=question,
    data_description=data_desc,
    input_data=[vmware_acquistion_summary],
    docset_summarizer=summarizer
)

In [ ]:
## Cell to answer the question tell me how Intuit is integrating Intuit Assist (their new AI offering) into existing products
intuit_prompt = JinjaPrompt(
    system="You are a helpful financial assistant.",
    user="""Here's an earnings call. Please answer the question {{ question }}
            {% for elt in doc.elements %}
            {{ elt.text_representation }}
            {% endfor %}""",
    question="Summarize how Intuit Assist is being integrated into Intuit's existing products"
)

def filterForAssist(elem: Element) -> bool:
  return "Assist" in elem.text_representation

intuit_assist_summary = (removed_orig_docset.filter(lambda doc: doc.properties['earnings_calls']['company_ticker'] == 'INTU')
                     .filter_elements(filterForAssist)
                     .llm_map(prompt=intuit_prompt, output_field="intuit_assist_summary", llm=oai, llm_mode=LLMMode.ASYNC))

In [ ]:
question = "Summarize how Intuit is integrating its AI assistant, Intuit Assist into its existing products. Give a 3 line summary for each quarter on the progress of the integration."
data_desc = "Intuit Assist integration"

oai = OpenAI(OpenAIModels.GPT_4O)
oaitk = OpenAITokenizer(OpenAIModels.GPT_4O.value.name, max_tokens=100_000)
summarizer = MultiStepDocumentSummarizer(
    llm=oai, 
    llm_mode=LLMMode.ASYNC, 
    question=question, 
    data_description=data_desc, 
    fields=['intuit_assist_summary'],
    tokenizer=oaitk
)
summarize_data(
    llm=oai,
    question=question,
    data_description=data_desc,
    input_data=[intuit_assist_summary],
    docset_summarizer=summarizer
)

In [ ]:
## Cell to answer the question 'Return all the companies that mentioned inflation and return a count of the number of times inflation was mentioned'
inflation_mentioned = (exploded_docset2.filter(lambda element: 'inflation' in element.text_representation.lower())
                     .groupby_count(field='properties.earnings_calls.company_name'))
inflation_mentioned.show()

In [ ]:
## Answering the same question with a LLMFilter. This is super slow. Need to figure out why. 
im = exploded_docset2.llm_filter(
    llm=oai,
    new_field="inflation_mentioned_confidence",
    prompt = LlmFilterMessagesJinjaPrompt.fork(filter_question="Does this text mention inflation?", use_elements=False),
    llm_mode = LLMMode.ASYNC
).groupby_count(field='properties.earnings_calls.company_name')
im.show()